In [ ]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig
from tqdm import tqdm

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel, sigma_gak
from kernels.reservoir import ReservoirKernel

from features.random_fourier import RBF_RandomFourierFeatures
from features.random_sig_fourier import TRP_RFSF_Gaussian, TRP_RFSF_Linear
from features.signature import sig, logsig
from features.random_warping_series import RandomWarpingSeries

In [ ]:
#################################
### Volterra Reservoir Kernel ###
#################################

def naive_gram(X, Y, tau, gamma):
    N, T, d = X.shape
    N2, T, d = Y.shape
    lin_ker = LinearKernel()
    state_space_gram = lin_ker(X, Y) #shape N1 N2 T
    prod = gamma**2 / (1 - tau**2 * state_space_gram)

    gram = torch.zeros(N, N2, dtype=X.dtype, device=X.device)
    for i in range(N):
        for j in range(N2):
            outer = 0
            for t in range(T):
                inner = 1
                for s in range(t+1):
                    inner *= prod[i, j, s]
                outer += inner
            gram[i, j] = 1 + outer
    return gram


def naive_gram2(X, Y, tau, gamma):
    N, T, d = X.shape
    N2, T, d = Y.shape
    lin_ker = LinearKernel()
    state_space_gram = lin_ker(X, Y) #shape N1 N2 T
    prod = gamma**2 / (1 - tau**2 * state_space_gram)

    gram = torch.zeros(N, N2, dtype=X.dtype, device=X.device)
    gram = gram + 1/(1-gamma**2)
    for t in range(T):
        gram = 1 + prod[:,:,T-1-t] * gram
    return gram



def volterra_reservoir_kernel_test():
    N = 3
    N2 = 2
    T = 70
    d = 2
    dtype = torch.float32
    torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach()
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach()
    tau = 0.9 / torch.maximum(X.abs().max(), Y.abs().max())
    print("tau", tau)
    gamma = 0.9

    ker = ReservoirKernel(tau=tau, gamma=gamma)
    gram = ker(X, Y)
    print("gram\n", gram)
    naive = naive_gram(X, Y, tau, gamma)
    print("naive\n", naive)
    naive2 = naive_gram2(X, Y, tau, gamma)
    print("naive2\n", naive2)
volterra_reservoir_kernel_test()

In [ ]:
# #######################
# ######### RWS #########
# #######################

def RWS_test():
    # Setup and parameters
    N = 3
    N2 = 2
    T = 70
    d = 2
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach()
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach()
    n_features = 20
    D_min=2
    D_max=5
    sigma=1

    # RWS
    rws = RandomWarpingSeries(n_features, D_min, D_max, sigma, LinearKernel(scale=1/d/T))
    feat_X = rws(X)
    feat_Y = rws(Y)
    K_trp = 1 + feat_X @ feat_Y.T
    print(feat_X)
    print(feat_X.shape)
    print(K_trp)

RWS_test()

In [ ]:
###############################
#### Lib signatory wrapper ####
###############################

def test_signatory():
    N,T,d = 2, 30, 2
    trunc_level = 3
    X = torch.randn(N, T, d)
    sigs = sig(X, trunc_level)
    logsigs = logsig(X, trunc_level)
    print("sigs", sigs.shape, sigs)
    print("logsigs", logsigs.shape, logsigs)

    # no batch case
    sigs0 = sig(X[0], trunc_level)
    logsigs0 = logsig(X[0], trunc_level)
    print("sigs0", sigs0.shape, sigs0)
    print("logsigs0", logsigs0.shape, logsigs0)
    
# test_signatory()

In [ ]:
##########################################################
#### Linear TRP-RFSF features  vs  Vanilla Sig kernel ####
##########################################################

def LINEAR_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = d
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(LinearKernel(), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = TRP_RFSF_Linear(trunc_level, n_features, only_last=True)
        feat_X = trp(X)
        feat_Y = trp(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

# LINEAR_trp_vs_kernel()
# # exact
# #  tensor([[ 4627.0444, -5744.9629],
# #         [ 9709.9150, -7802.3965],
# #         [-1727.5579,  2563.8979]], device='cuda:0')
# # 100%|██████████| 10000/10000 [00:15<00:00, 641.01it/s] # n_features = 500, trunc_level = 5
# # mean
# #  tensor([[ 4624.3896, -5737.0781],
# #         [ 9733.1250, -7796.5479],
# #         [-1733.9088,  2575.4778]], device='cuda:0')

In [ ]:
#######################################################
#### Gaussian TRP-RFSF features  vs  SigRBF kernel ####
#######################################################

def GAUSSIAN_trp_vs_kernel():
    #parameters
    N = 3
    N2 = 2
    T = 20
    d = 2
    trunc_level = 5
    n_features = 5000
    sigma = 1.0
    dtype = torch.float32
    #torch.manual_seed(3)
    X = torch.randn(N, T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)
    Y = torch.randn(N2,T, d, dtype=dtype).to("cuda").detach() / np.sqrt(d)

    #exact sig kernel
    sigker = TruncSigKernel(RBFKernel(sigma=sigma), normalize=False, trunc_level=trunc_level, geo_order=1, max_batch=50000)
    K = sigker(X,Y)
    print("exact\n", K)

    #trp
    MC_iter = 10000
    res = []
    for i in tqdm(range(MC_iter)):
        trp = TRP_RFSF_Gaussian(trunc_level, n_features, sigma, only_last=True)
        feat_X = trp(X)
        feat_Y = trp(Y)
        K_trp = 1 + feat_X @ feat_Y.T
        res.append(K_trp)
    res = torch.stack(res)
    example = res[0]
    mean = res.mean(dim=0)
    print("mean\n", mean)
    print("example\n", example)

# GAUSSIAN_trp_vs_kernel()
# # exact
# #  tensor([[ -8.7953, -30.1428],
# #         [ -4.1622,  -4.5515],
# #         [-25.4650,  42.8830]], device='cuda:0')
# # 100%|██████████| 10000/10000 [05:55<00:00, 28.10it/s] # n_features=5000, trunc_level=5
# # mean
# #  tensor([[ -9.9236, -30.5771],
# #         [ -4.4291,  -4.5940],
# #         [-25.3243,  43.3625]], device='cuda:0')

In [ ]:
############################################################
#### Test RBF_RandomFourierFeatures vs exact RBF kernel ####
############################################################
def rff_vs_exact_RBFKernel():
    N=3
    N2= 2
    d = 10
    sigma=1
    dtype = torch.float64
    # torch.manual_seed(1)
    X = torch.randn(N, d, dtype=dtype).to("cuda") /np.sqrt(d)
    Y = torch.randn(N2, d, dtype=dtype).to("cuda") / np.sqrt(d)

    # Exact RBF kernel
    k = RBFKernel(sigma=sigma)
    K = k(X, Y)

    # Approximate RBF kernel using RBF_RandomFourierFeatures
    N_MC = 10000
    res = []
    for i in range(N_MC):
        RFF = RBF_RandomFourierFeatures(n_features=1000,
                                        sigma=sigma,
                                        method="cos(x)sin(x)",
                                        # method = "cos(x + b)",
                                        )
        feat_X = RFF(X)
        feat_Y = RFF(Y)
        K_rff = feat_X @ feat_Y.T
        res.append(K_rff)
    K_rff = torch.mean(torch.stack(res), dim=0)

    print("K\n",K)
    print("K_rff\n",K_rff)
    print("diff\n", K-K_rff)
    print("diffmean\n", torch.mean(abs(K-K_rff)))
    # the RFF approach cant reproduce results smaller than 1e-5 for some reason
    
# rff_vs_exact_RBFKernel()

In [ ]:
#####################################
######### FROM KSIG LIBRARY #########
#####################################
import numpy as np
import ksig

def ksig_readme():
    # Number of signature levels to use.
    n_levels = 5 
    normalize=False

    # Use 100 components in RFF and projection.
    n_components = 100

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    # Instantiate tensor random projections.
    proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    rfsf_trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, static_features=static_feat, projection=proj, normalize=normalize)

    # Generate 1000 sequences of length 200 with 100 features.
    n_seq, l_seq, n_feat = 3, 20, 10
    X = np.random.randn(n_seq, l_seq, n_feat) / np.sqrt(n_feat)

    # Fit the kernel to the data.
    rfsf_trp_kernel.fit(X)

    # Compute the kernel matrix as before.
    K_XX = rfsf_trp_kernel(X)  # K_XX has shape (1000, 1000).

    # GEnerate another array of 800 sequences of length 250 and 100 features.
    n_seq2, l_seq2 = 4, 20
    Y = np.random.randn(n_seq2, l_seq2, n_feat) / np.sqrt(n_feat)

    # Compute the kernel matrix between X and Y.
    # The kernel does not have to be fitted a second time.
    K_XY = rfsf_trp_kernel(X, Y)  # K_XY has shape (1000, 800)

    # Alternatively, we may compute features separately for X and Y. Under the hood,
    # this is what the call method does, i.e. compute features and take their inner product.
    P_X = rfsf_trp_kernel.transform(X)  # P_X has shape (1000, 501)
    P_Y = rfsf_trp_kernel.transform(Y)  # P_Y shape shape (800, 501)
    print("P_X", P_X)






    # Use the RBF kernel for vector-valued data as static (base) kernel.
    static_kernel = ksig.static.kernels.RBFKernel()
    order = 1
    sig_kernel = ksig.kernels.SignatureKernel(n_levels=n_levels, order=order, static_kernel=static_kernel,
                                              normalize=normalize)

    # Sequence kernels take as input an array of sequences of ndim == 3,
    # and work as a callable for computing the kernel matrix.
    K_XX_exact = sig_kernel(X)
    K_XY_exact = sig_kernel(X, Y)  
    print("\n\nTRP XX\n", K_XX)
    print("Exact XX\n", K_XX_exact)
    print("\n\nTRP XY\n", K_XY)
    print("Exact XY\n", K_XY_exact)

# ksig_readme()

In [ ]:
#########################
## KSIG test TRP vs DP ##
#########################
import ksig
import numpy as np

def trp_vs_dp():
    n_seq, l_seq, n_feat = 10, 150, 100
    X = np.random.randn(n_seq, l_seq, n_feat)
    n_levels = 10  # Number of signature levels to use.
    n_components = 100 # Use dimension of RFF map

    # Instantiate RFF feature map.
    static_feat = ksig.static.features.RandomFourierFeatures(n_components=n_components)
    trp_proj = ksig.projections.TensorizedRandomProjection(n_components=n_components)
    dp_proj = ksig.projections.DiagonalProjection()

    # The RFSF-TRP feature map and kernel. Additionally to working as a callable for
    # computing a kernel, it implements a fit and a transform method.
    trp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels, 
        static_features=static_feat, 
        projection=trp_proj
        )
    dp_kernel = ksig.kernels.SignatureFeatures(
        n_levels=n_levels,
        static_features=static_feat,
        projection=dp_proj
        )
    trp_kernel.fit(X)
    dp_kernel.fit(X)


    # TIME IT
    import timeit
    def function_dp():
        dp_kernel(X, X)
    def function_trp():
        trp_kernel(X, X)
    execution_time_dp = timeit.timeit(function_dp, number=10)
    print("Execution time of dp\t:", execution_time_dp)
    execution_time_trp = timeit.timeit(function_trp, number=10)
    print("Execution time of trp\t:", execution_time_trp)
    
#trp_vs_dp()